In [2]:
import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os

In [58]:
"""
The following constants are defined to specify the three export files and the path. 
Adjust the path to the location on your computer that holds the Wikipedia articles XML dump.
"""

PATH_WIKI_XML = '../data/'
FILENAME_WIKI = 'rowiki-20140729-corpus.xml'
FILENAME_WIKI = 'rowiki-20171120-pages-articles-multistream.xml'
FILENAME_ARTICLES = 'articles.csv'
FILENAME_REDIRECT = 'articles_redirect.csv'
FILENAME_TEMPLATE = 'articles_template.csv'
ENCODING = "utf-8"

In [59]:
pathWikiXML = os.path.join(PATH_WIKI_XML, FILENAME_WIKI)
pathArticles = os.path.join(PATH_WIKI_XML, FILENAME_ARTICLES)
pathArticlesRedirect = os.path.join(PATH_WIKI_XML, FILENAME_REDIRECT)
pathTemplateRedirect = os.path.join(PATH_WIKI_XML, FILENAME_TEMPLATE)


In [60]:
totalCount = 0
articleCount = 0
redirectCount = 0
templateCount = 0
title = None
start_time = time.time()

In [61]:
def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

In [65]:
limitEvents =200
countEvents = 0

with codecs.open(pathArticles, "w", ENCODING) as articlesFH, \
        codecs.open(pathArticlesRedirect, "w", ENCODING) as redirectFH, \
        codecs.open(pathTemplateRedirect, "w", ENCODING) as templateFH:
    articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)
    redirectWriter = csv.writer(redirectFH, quoting=csv.QUOTE_MINIMAL)
    templateWriter = csv.writer(templateFH, quoting=csv.QUOTE_MINIMAL)
    articlesWriter.writerow(['id', 'title', 'redirect'])
    redirectWriter.writerow(['id', 'title', 'redirect'])
    templateWriter.writerow(['id', 'title'])
    
    for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
    
        # limit events processed
        countEvents += 1
        if countEvents >= limitEvents:
            print(f"stopping after {countEvents} events")
            break


        tname = strip_tag_name(elem.tag)
        #print(f"event = {event}, tname = {tname}:{elem}")
        if event == 'start':
            if tname == 'page':
                title = ''
                id = -1
                redirect = ''
                inrevision = False
                ns = 0
            elif tname == 'revision':
                # Do not pick up on revision id's
                inrevision = True
        else:
            if tname == 'title':
                title = elem.text
            elif tname == 'id' and not inrevision:
    #             id = int(elem.text)
                id = elem.text
            elif tname == 'redirect':
                pass
                # redirect = elem.attrib['title']
                    
            elif tname == 'ns':
                ns = elem.text
                #ns = int(elem.text)

            elif tname == 'text':
                text = elem.text
                print(f"event = {event}, tname = {tname} text={text}")
            elif tname == 'page':
                totalCount += 1
                if ns == 10:
                    templateCount += 1
                    templateWriter.writerow([id, title])
                elif len(redirect) > 0:
                    articleCount += 1
                    articlesWriter.writerow([id, title, redirect])
                else:
                    redirectCount += 1
                    redirectWriter.writerow([id, title, redirect])
        if totalCount > 1 and (totalCount % 100000) == 0:
            print("{:,}".format(totalCount))
        elem.clear()

    elapsed_time = time.time() - start_time
    print("Total pages: {:,}".format(totalCount))
    print("Template pages: {:,}".format(templateCount))
    print("Article pages: {:,}".format(articleCount))
    print("Redirect pages: {:,}".format(redirectCount))
    print("Elapsed time: {}".format(hms_string(elapsed_time)))

event = end, tname = text text=None
event = end, tname = text text=None
event = end, tname = text text={{trimite|IRIS|alte sensuri|Iris (dezambiguizare)}}
{{Infocaseta Muzician
| Nume                = Iris
| Img                 = Mihai Alexandru, Nelu Dumitrescu, Eugen Brebu, Valter Popa, Cristian Minculescu, Doru Brobeica.jpg
| Img_desc            = 
| Img_mărime          = 
| Peisaj              = 
| Culoare             = Grup sau orchestră
| Origine             = [[Fișier:Flag of Romania.svg|20px]] [[București]], [[România]]
| Gen                 = [[rock|rock clasic]]{{·}}[[hard rock]]{{·}}[[pop rock]]{{·}}[[blues rock]]{{·}}[[heavy metal]]{{·}}[[speed metal]]{{·}}[[rock simfonic]]
| Ani activi          = [[1976]] – prezent
| Casă de discuri     = [[Electrecord]] (1977-1996)<br>Zone Records (1996-2006)<br>[[Roton]] (2006-2012)<br>[[Cat Music]] (2012-prezent)
| Interpretare cu     = 
| URL                 = http://www.trupairis.ro/
| Membri              = [[Costică Sandu]]<br>[[Ion 

In [21]:
pwd()

'/Users/alexlfll/Documents/projects.nosync/diacritic/setup'